In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,PreTrainedTokenizerFast
import torch


# Specify the directory containing the model and tokenizer files
model_dir = './model/bart1/checkpoint-133500'

# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(model_dir)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:
# 문장에 사용된 특수 문자를 정리하는 함수
def clean_special_characters(text):
    special_characters = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    cleaned_text = ''.join(char for char in text if char not in special_characters)
    return cleaned_text

# 입력 문장
satoori_text = """전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.

"""

# 입력 문장 정리
cleaned_satoori_text = clean_special_characters(satoori_text)

# Tokenize the cleaned input text
input_ids = tokenizer(cleaned_satoori_text, return_tensors='pt').input_ids

# Generate the translation
translation_ids = model.generate(input_ids, max_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)

# Decode the generated translation
translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)

# Print the translated text
print("번역 결과:", translated_text)


번역 결과: 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의


In [9]:
import nltk.translate.bleu_score as bleu
#라이브러리 총집합
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [10]:
data = pd.read_csv('./document_summaries_test_utf.csv')

In [11]:
device = torch.device("cuda")

In [12]:
# 1열(첫 번째 열)에서 가장 긴 문자열의 길이 찾기
max_length = data.iloc[:, 0].str.len().max()

# 가장 긴 문자열 찾기
longest_sentence = data.loc[data.iloc[:, 0].str.len() == max_length, data.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 1630
가장 긴 문장을 출력함: 트럼프 대통령은 특히 협상이 타결된다면 미 워싱턴DC에서 시진핑 중국 국가주석과 미중 정상회담을 개최하겠다고 해, 이르면 이달 내 미중 정상이 만나 협상 타결을 선언할 수 있게 될지 주목된다. 트럼프 대통령은 이날 백악관에서 9번째 미중 통상협상에 참여한 류허 중국 부총리와 면담한 자리에서 협상 전망을 묻는 기자 질문에 "먼저 무역합의부터 타결 짓고 시진핑 주석과 정상회담을 가질 것"이라면서 "우리는 아마도 4주 안에 알게 될 것이며, (전망이) 매우 좋아 보인다"고 말했다. 이는 당초 트럼프 대통령이 미중 정상회담 일정을 발표할 것이란 예측을 깨고 '선 협상타결 후 정상회담' 전략을 고수한 것이다. 트럼프 대통령은 미중정상회담 장소는 워싱턴DC가 될 것이라고 밝혔다고 로이터통신이 전했다. 트럼프 대통령은 이어 "우리는 중국과의 무역에서 한해에 최대 6000억달러까지 수년을 잃어왔기 때문에 좋은 딜을 타결해야 한다"며 굿 딜이 아니면 노딜이 될 수밖에 없음을 강조했다. 트럼프 대통령은 그러나 "중국이 강력하게 합의를 원하고 있고 큰 진전을 이뤄왔다"며서 머지않아 최종 거래를 성사시킬 수 있을 것으로 기대했다. 트럼프 대통령은 또 "갈 길이 남아있다"면서도 협상이 타결된다면 그것은 "기념비적 합의"가 될 것이라고 말했다. 트럼프 대통령은 진전이 빠른 속도로 이뤄지고 있으며, 갈 길이 남아있지만 그다지 먼 길이 남아있는 것은 아니라고 말했다. 트럼프 대통령은 남은 협상 난제가 뭐냐는 질문을 받고는 '관세'와 '지적 재산권 탈취', '합의 이행' 문제를 꼽으며 류 부총리와 관세 문제를 논의할 것이라고 말했다. 류 부총리도 "협상에서 엄청난 진전이 있었다"고 밝혔다. 월스트리트 저널을 비롯한 미국 주요 언론은 미국과 중국이 9번이나 해온 마라톤 협상에서 거의 모든 쟁점들에서 합의에 도달했고 마지막 쟁점만 해결하면 양 정상이 만나 최종 타결하는 즉시 무역전쟁 끝내는 엔드게임에 돌입한 것으로 해석하고 있다. 아직

In [15]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

# sumDataset 클래스 정의
class evalDataset(Dataset):
    def __init__(self, original, extractive, tokenizer):
        self.original = original
        self.extractive = extractive
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.original)


    def __getitem__(self, idx):
        original_text = self.original[idx]
        extractive_text = self.extractive[idx]

        # 토크나이징 및 패딩 수행
        inputs = self.tokenizer(
            text=original_text,
            padding="max_length",  # 필요한 경우 max_length 설정
            truncation=True,       # 필요한 경우 문장 자르기 설정
            return_tensors="pt",   # PyTorch 텐서로 반환
            max_length=512,
            )

        outputs = self.tokenizer(
            text=extractive_text,
            padding="max_length",  # 필요한 경우 max_length 설정
            truncation=True,       # 필요한 경우 문장 자르기 설정
            return_tensors="pt",   # PyTorch 텐서로 반환
            max_length=512,
            )

        # 모델 입력을 포함하는 딕셔너리 반환
        sample = {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': outputs.input_ids.squeeze()  # 예시로 labels에 input_ids를 사용하도록 설정
        }

        return sample

In [22]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
# 테스트 데이터 전처리

# 1열(standard)과 2열(dialect)을 각각 features(X)와 labels(y)로 설정
x = data['original']
y = data['extractive']

test_dataset = evalDataset(x, y, tokenizer)

In [24]:
print(x)

0        문 대통령은 "5G가 각 분야에 융합되면, 정보통신산업을 넘어 자동차, 드론(무인항...
1        손 대표는 회의를 주재하면서 "의원들이나 지역위원장들, 당원들이 다음 선거에 대해 ...
2        무대에 마련된 초고화질(4K UHD) 대형 스크린에서 국악기와 양악기가 함께 연주하...
3        박 시장의 발언은 서울 내 노후 아파트 주민들이 서울시를 상대로 재건축 인허가를 요...
4        SK와 알파벳은 신규 투자는 지주회사가 전담하고, 자회사는 본업에만 충실할 수 있도...
                               ...                        
29311    외국인 계절근로자들의 지역 체류와 근로의욕 고취를 위한 취지다. 베트남 국적의 근로...
29312    국립과천과학관에서도 다음달 5일부터 '우주연구실 인턴 체험 특별전'을 연다. 대전에...
29313    황 대표는 이날 김광림 최고위원과 영주시민회관에서 열린 한국당 영주문경예천 당원협의...
29314    특히 경북형 사회적 경제 청년일자리사업 2억6700만 원, 사회적기업 사업개발비 3...
29315    이들 4개 지구는 지난해 9월 수도권 30만 가구 주택공급 계획과 함께 1차 입지로...
Name: original, Length: 29316, dtype: object


In [25]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    output_dir="./testoutput/bart1",
    predict_with_generate=True,

)


# Trainer 객체 생성 (훈련 부분 제거)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    eval_dataset = test_dataset,
)

# 테스트 데이터를 평가 데이터셋으로 설정
test_dataset = evalDataset(x, y ,tokenizer)

In [26]:
# 첫 번째 샘플 얻기
first_sample = test_dataset[0]
print(first_sample)

{'input_ids': tensor([14111, 15606, 14050,   252,   270,  8981, 14319, 22331, 20550, 15848,
          243, 23700, 14934, 12007, 15409, 15527,   243, 14340, 10341,   239,
        10667, 12037, 16732,  9264, 14505, 20760,   243, 25788, 13679, 19541,
        23937, 16152, 16794,   266, 16152, 15087, 16464, 20680,  9120, 20094,
        11786, 23107, 14199, 14982, 21493, 28897, 15929, 17554, 11280, 14416,
        14345, 19810, 14253, 14130, 14111, 15606, 27274, 24533, 16956, 25019,
        14144,   270, 20527, 15093, 14508, 28451, 13842, 14490, 13607, 16785,
        14990, 14511, 19754, 14111, 15606, 14370, 15592, 25053, 16745, 21039,
        14199, 14144,   270, 18087, 20779, 22112, 15911,  9776, 14318, 14047,
        12669, 15761, 19754,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3, 

In [27]:
# 평가 수행
results = trainer.evaluate()

# 평가 결과 출력
print(results)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hjs3545 (nlp_learning). Use `wandb login --relogin` to force relogin


{'eval_loss': 0.20944644510746002, 'eval_rouge1': 0.1953, 'eval_rouge2': 0.061, 'eval_rougeL': 0.1927, 'eval_rougeLsum': 0.1927, 'eval_gen_len': 19.9236, 'eval_runtime': 811.0791, 'eval_samples_per_second': 36.144, 'eval_steps_per_second': 4.519}
